In [1]:
import numpy as np
from scipy import linalg

# 1. Band Matrix Form (SciPy)


$$
\begin{bmatrix}
\colorbox{pink}{$a_{00}$} & \colorbox{cyan}{$a_{01}$} & \colorbox{yellow}{$a_{02}$} & 0 & 0 \\
\colorbox{green}{$a_{10}$} & \colorbox{pink}{$a_{11}$} & \colorbox{cyan}{$a_{12}$} & \colorbox{yellow}{$a_{13}$} & 0 \\
0 & \colorbox{green}{$a_{21}$} & \colorbox{pink}{$a_{22}$} & \colorbox{cyan}{$a_{23}$} & \colorbox{yellow}{$a_{24}$} \\
0 & 0 & \colorbox{green}{$a_{32}$} & \colorbox{pink}{$a_{33}$} & \colorbox{cyan}{$a_{34}$} \\
0 & 0 & 0 & \colorbox{green}{$a_{43}$} & \colorbox{pink}{$a_{44}$}
\end{bmatrix}
$$
- Lower Band Width: 1
- Upper Band Width: 2

***SciPy Band Matrix Format***

$$
\begin{bmatrix}
0 & 0 & \colorbox{yellow}{$a_{02}$} & \colorbox{yellow}{$a_{13}$} & \colorbox{yellow}{$a_{24}$} \\
0 & \colorbox{cyan}{$a_{01}$} & \colorbox{cyan}{$a_{12}$} & \colorbox{cyan}{$a_{23}$} & \colorbox{cyan}{$a_{34}$} \\
\colorbox{pink}{$a_{00}$} & \colorbox{pink}{$a_{11}$} & \colorbox{pink}{$a_{22}$} & \colorbox{pink}{$a_{33}$} & \colorbox{pink}{$a_{44}$} \\
\colorbox{green}{$a_{10}$} & \colorbox{green}{$a_{21}$} & \colorbox{green}{$a_{32}$} & \colorbox{green}{$a_{43}$} & 0 \\
0 & 0 & 0 & 0 & 0
\end{bmatrix}
$$

- Maintains column index and stores only the band in a flattened horizontal format.
- Band width $\ll n$: Useful for memory and computation efficiency. (from 5 x 5 matrix to 4 x 5 matrix)

example

$$
\begin{bmatrix}
\colorbox{pink}{$1$} & \colorbox{cyan}{$2$} & 0 & 0 & 0 \\
\colorbox{green}{$1$} & \colorbox{pink}{$4$} & \colorbox{cyan}{$1$} & 0 & 0 \\
\colorbox{green}{$5$} & \colorbox{green}{$0$} & \colorbox{pink}{$1$} & \colorbox{cyan}{$2$} & 0 \\
0 & \colorbox{green}{$1$} & \colorbox{green}{$2$} & \colorbox{pink}{$2$} & \colorbox{cyan}{$1$} \\
0 & 0 & \colorbox{green}{$2$} & \colorbox{green}{$1$} & \colorbox{pink}{$1$}
\end{bmatrix}
$$

$$
\begin{bmatrix}
0 & \colorbox{cyan}{$2$} & \colorbox{cyan}{$1$} & \colorbox{cyan}{$2$} & \colorbox{cyan}{$1$} \\  % Upper band (k = 1)
\colorbox{pink}{$1$} & \colorbox{pink}{$4$} & \colorbox{pink}{$1$} & \colorbox{pink}{$2$} & \colorbox{pink}{$1$} \\  % Main diagonal (k = 0)
\colorbox{green}{$1$} & \colorbox{green}{$0$} & \colorbox{green}{$2$} & \colorbox{green}{$1$} & 0 \\  % Lower band (k = -1)
\colorbox{green}{$5$} & \colorbox{green}{$1$} & \colorbox{green}{$2$} & 0 & 0 \\  % Lower band (k = -2)
\end{bmatrix}
$$

In [2]:
A = np.array([
    [1, 2, 0, 0, 0],
    [1, 4, 1, 0, 0],
    [5, 0, 1, 2, 0],
    [0, 1, 2, 2, 1],
    [0, 0, 2, 1, 1]
])

In [3]:
# Define the lower and upper bandwidths
low_bdw = 2
upp_bdw = 1

# Create the band matrix
# n = num of bands
n = upp_bdw + low_bdw + 1
band_A = np.zeros((n, A.shape[1]), dtype = np.float64)

band_A[-1, : -2] = np.diag(A, k = -2)
band_A[-2, : -1] = np.diag(A, k = -1)
band_A[-3, : ] = np.diag(A, k = 0)
band_A[-4, : -1] = np.diag(A, k = 1)
band_A

array([[2., 1., 2., 1., 0.],
       [1., 4., 1., 2., 1.],
       [1., 0., 2., 1., 0.],
       [5., 1., 2., 0., 0.]])

## 2. Band Matrix Solver ($A\mathbf{x} = \mathbf{b}$)

```python
x = linalg.slove_banded((lbw, ubw), band_A, b)
```

* band_A is a band matrix
* algorithms
  1. LU decomposition   (Lapack: gbsv)
  2. tridiagonal solver (Lapack: gtsv)

$$
b = 
\begin{bmatrix}
1 \\
1 \\
1 \\
1 \\
1
\end{bmatrix}, \quad
A_1 = 
\begin{bmatrix}
1 & 2 & 0 & 0 & 0 \\
1 & 4 & 1 & 0 & 0 \\
5 & 0 & 1 & 2 & 0 \\
0 & 1 & 2 & 2 & 1 \\
0 & 0 & 2 & 1 & 1
\end{bmatrix}, \quad
A_2 = 
\begin{bmatrix}
2 & 1 & 0 & 0 & 0 \\
1 & 2 & 1 & 0 & 0 \\
0 & 1 & 2 & 1 & 0 \\
0 & 0 & 1 & 2 & 1 \\
0 & 0 & 0 & 1 & 2
\end{bmatrix}
\tilde{A_1} = 
\begin{bmatrix}
0 & 2 & 1 & 2 & 1 \\
1 & 4 & 1 & 2 & 1 \\
1 & 0 & 2 & 1 & 0 \\
5 & 1 & 2 & 0 & 0
\end{bmatrix}, \quad
\tilde{A_2} = 
\begin{bmatrix}
0 & 1 & 1 & 1 & 1 \\
2 & 2 & 2 & 2 & 2 \\
1 & 1 & 1 & 1 & 0
\end{bmatrix}
$$

In [4]:
# Vector b
b = np.array([1, 1, 1, 1, 1])

# Matrix A (first one)
# we have no access to it
A1 = np.array([
    [1, 2, 0, 0, 0],
    [1, 4, 1, 0, 0],
    [5, 0, 1, 2, 0],
    [0, 1, 2, 2, 1],
    [0, 0, 2, 1, 1]
])

# Matrix A (second one)
# we have no access to it
A2 = np.array([
    [2, 1, 0, 0, 0],
    [1, 2, 1, 0, 0],
    [0, 1, 2, 1, 0],
    [0, 0, 1, 2, 1],
    [0, 0, 0, 1, 2]
])

# Matrix A_tilde (first one)
A_band1 = np.array([
    [0, 2, 1, 2, 1],
    [1, 4, 1, 2, 1],
    [1, 0, 2, 1, 0],
    [5, 1, 2, 0, 0]
])

# Matrix A_tilde (second one)
A_band2 = np.array([
    [0, 1, 1, 1, 1],
    [2, 2, 2, 2, 2],
    [1, 1, 1, 1, 0]
])

In [5]:
x1 = linalg.solve_banded((2, 1), A_band1, b)
x2 = linalg.solve_banded((1, 1), A_band2, b)

In [6]:
print(x1)

[ 0.42857143  0.28571429 -0.57142857 -0.28571429  2.42857143]


In [7]:
print(x2)

[0.5 0.  0.5 0.  0.5]


### How to check the accuracy? ($A_{\text{band}}\mathbf{x} \neq \mathbf{b}$)

1. Convert the band matrix into the original matrix and compare $A\mathbf{x}$ and $\mathbf{b}$ $\rightarrow$ but then, why did we get the band matrix in the first place? (BAD IDEA; defeats the purpose of saving memory space)
2. Use custom function to manually compute $A\mathbf{x}$ using only the band matrix and compare it with $\mathbf{b}$ using ```np.allclose``` 

# 3. Band Matrix Solver for Positive Definite Matrices ($A\mathbf{x} = \mathbf{b}$)

For symmetric and Hermitiain matrices:

```python
x = linalg.solveh_banded(band_a_h, b, lower = False)
```

* only one side of bands are required
* algorithms
  1. Cholesky decomposition       (Lapack: pbsv)
  2. $LDL^\top$ decomposition     (Lapack: ptsv)
  3. $U^\top DU$ decomposition    (for tridiagonals)

$$
\text{Positive Definite Matrix:}
\begin{bmatrix}
2 & 1 & 0 & 0 & 0 \\
1 & 2 & 1 & 0 & 0 \\
0 & 1 & 2 & 1 & 0 \\
0 & 0 & 1 & 2 & 1 \\
0 & 0 & 0 & 1 & 2
\end{bmatrix} \rightarrow
\text{Upper Form:} \quad
\begin{bmatrix}
0 & \colorbox{black!20}{$1$} & \colorbox{black!20}{$1$} & \colorbox{black!20}{$1$} & \colorbox{black!20}{$1$} \\
\colorbox{pink}{$2$} & \colorbox{pink}{$2$} & \colorbox{pink}{$2$} & \colorbox{pink}{$2$} & \colorbox{pink}{$2$}
\end{bmatrix} \text{or   }
\text{Lower Form:} \quad
\begin{bmatrix}
\colorbox{pink}{$2$} & \colorbox{pink}{$2$} & \colorbox{pink}{$2$} & \colorbox{pink}{$2$} & \colorbox{pink}{$2$} \\
\colorbox{black!20}{$1$} & \colorbox{black!20}{$1$} & \colorbox{black!20}{$1$} & \colorbox{black!20}{$1$} & 0
\end{bmatrix}
$$

Example:

$$
A_1 = 
\begin{bmatrix}
8 & 2-i & 0 & 0 \\
2+i & 5 & i & 0 \\
0 & -i & 9 & -2-i \\
0 & 0 & -2+i & 6
\end{bmatrix}, \quad
\tilde{A}_1 = 
\begin{bmatrix}
0 & 2-i & i & -2-i \\
8 & 5 & 9 & 6
\end{bmatrix}, \quad
b = 
\begin{bmatrix}
1 \\
1 \\
1 \\
1
\end{bmatrix}
$$

In [8]:
# Define A1 (full matrix)
# we have no access to it
A1 = np.array([
    [8, 2-1j, 0, 0],
    [2+1j, 5, 1j, 0],
    [0, -1j, 9, -2-1j],
    [0, 0, -2+1j, 6]
], dtype=np.complex128)

# Define A1_band (band matrix format)
A1_band = np.array([
    [0, 2-1j, 1j, -2-1j],  # Upper bands
    [8, 5, 9, 6]           # Diagonal
], dtype=np.complex128)

# Define b (right-hand side vector)
b1 = np.array([1, 1, 1, 1], dtype=np.float64)

In [9]:
x1 = linalg.solveh_banded(A1_band, b1, lower = False) # upper form
print(x1)

[0.08818236+0.03959208j 0.18116377-0.06778644j 0.17156569+0.04259148j
 0.23095381-0.01439712j]


$$
A_2 = 
\begin{bmatrix}
2 & 1 & 0 & 0 & 0 \\
1 & 2 & 1 & 0 & 0 \\
0 & 1 & 2 & 1 & 0 \\
0 & 0 & 1 & 2 & 1 \\
0 & 0 & 0 & 1 & 2
\end{bmatrix}, \quad
\tilde{A}_2 = 
\begin{bmatrix}
0 & 1 & 1 & 1 & 1 \\
2 & 2 & 2 & 2 & 2
\end{bmatrix}, \quad
b = 
\begin{bmatrix}
1 \\
1 \\
1 \\
1 \\
1 \\
\end{bmatrix}
$$

In [10]:
# Define A2 (full matrix)
# we have no access to it
A2 = np.array([
    [2, 1, 0, 0, 0],
    [1, 2, 1, 0, 0],
    [0, 1, 2, 1, 0],
    [0, 0, 1, 2, 1],
    [0, 0, 0, 1, 2]
], dtype=np.float64)

# Define A2_band (band matrix format)
A2_band = np.array([
    [0, 1, 1, 1, 1],  # Upper band
    [2, 2, 2, 2, 2]   # Diagonal
], dtype=np.float64)

# Define b (row form)
b2 = np.array([1, 1, 1, 1, 1], dtype=np.float64)

In [11]:
x2 = linalg.solveh_banded(A2_band, b2, lower = False) # upper form
print(x2)

[0.5 0.  0.5 0.  0.5]


### How to check the accuracy? ($A_{\text{band}}\mathbf{x} \neq \mathbf{b}$)

The same issue exists here as well. Refer to section 1.2

# 4. Case for a Band Matrix Solver

### Tridiagonal Matrix and Solvers

$$
A = 
\begin{bmatrix}
2 & 1 & 0 & \cdots & 0 \\
1 & 2 & 1 & \ddots & \vdots \\
0 & 1 & 2 & \ddots & 0 \\
\vdots & \ddots & \ddots & \ddots & 1 \\
0 & \cdots & 0 & 1 & 2
\end{bmatrix}, \quad
b = 
\begin{bmatrix}
1 \\
1 \\
\vdots \\
1 \\
1
\end{bmatrix}
$$

**Matrix size:** $10000 \times 10000$ \
**Vector size:** $10000$

#### Memory Requirements:
- For **dense** $A$: **760 Mb**  
- For **band matrix** $A$: **156 Kb**

---

### Solvers and Performance:

1. Dense solver:
   $$
   \texttt{solve}(A, b)
   \begin{cases}
   3.36 \, \text{sec} & \text{assume\_a = "pos"} \\
   3.97 \, \text{sec} & \text{assume\_a = "gen"}
   \end{cases}
   $$

2. Banded solvers:
   - $\texttt{solveh\_banded(band\_a\_h, b)}$: **0.00023 sec**  
   - $\texttt{solve\_banded((1, 1), band\_a, b)}$: **0.00029 sec**